In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
import catboost as cbt
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
import optiver2023
import warnings
from warnings import simplefilter
warnings.filterwarnings('ignore')
simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [3]:
df = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [4]:
print(sum(df['target'].isna()))
df['target'].describe()

88


count    5.237892e+06
mean    -4.756125e-02
std      9.452860e+00
min     -3.852898e+02
25%     -4.559755e+00
50%     -6.020069e-02
75%      4.409552e+00
max      4.460704e+02
Name: target, dtype: float64

In [5]:
df.dropna(subset=['target'], inplace=True)

In [6]:
# Feature Engineering function
def generate_features(df):
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag', 'imbalance_size', 'matched_size',
                'bid_size', 'ask_size', 'reference_price', 'far_price', 'near_price', 'ask_price',
                'bid_price', 'wap', 'imb_s1', 'imb_s2']
    
    # Imbalance features
    df['imb_s1'] = (df['bid_size'] - df['ask_size']) / (df['bid_size'] + df['ask_size'])
    df['imb_s2'] = (df['imbalance_size'] - df['matched_size']) / (df['matched_size'] + df['imbalance_size'])
    
    prices = ['reference_price', 'far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for i, a in enumerate(prices):
        for j, b in enumerate(prices[i+1:], i+1):
            df[f'{a}_{b}_diff'] = df[a] - df[b]
            features.append(f'{a}_{b}_diff')
            df[f'{a}_{b}_delta'] = (df[a] - df[b]) / (df[a] + df[b])
            features.append(f'{a}_{b}_delta')
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1)
                    min_ = df[[a,b,c]].min(axis=1)
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_

                    df[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
                    features.append(f'{a}_{b}_{c}_imb2')
    
    # usual stats
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 4, 8, 16]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window, fill_method=None)
    
    df, add_features = additional_features(df)
    
    feature_names = [w for w in df.columns if w not in ["row_id", "time_id", "target", "currently_scored"]]
    
    return df, feature_names

def additional_features(df):
    add_features = ['signed_imb_size', 'mid_price', 'volume', 'imb_ratio', 'size_imbalance', 'imb_bid_r',
                    'imb_ask_r', 'mat_bid_ask_r', "imbalance_momentum", "spread_intensity", 'price_pressure',
                    'market_urgency']
    df['signed_imb_size'] = df['imbalance_buy_sell_flag'] * df['imbalance_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['volume'] = df['bid_size'] + df['ask_size']
    df['imb_ratio'] = df['imbalance_size'] / df['matched_size']
    df['size_imbalance'] = df['bid_size']/df['ask_size']
    df['imb_bid_r'] = df['imbalance_size']/df['bid_size']
    df['imb_ask_r'] = df['imbalance_size']/df['ask_size']
    df['mat_bid_ask_r'] = df.eval('matched_size/(bid_size+ask_size)')
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["spread_intensity"] = df.groupby(['stock_id'])['ask_price_bid_price_diff'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['ask_price_bid_price_diff'] * df['imb_s1']
    df['delta_bid_ask_size'] = (df['bid_size'] - df['ask_size']) / (df['bid_size'] + df['ask_size'] + 1)
    df['bid_ask_matched_r'] = (df['bid_size'] - df['ask_size']) / (df['matched_size'] + 1)
    
    prices = ['reference_price', 'far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    # Discrete derivatives
    for a in prices:
        df[f'{a}_first_derivative'] = df.groupby(['stock_id'])[a].diff()
    for a in sizes:
        df[f'{a}_first_derivative'] = df.groupby(['stock_id'])[a].diff()
    
    return df, add_features

In [7]:
df_train, feature_names = generate_features(df)
X = df_train[feature_names].values
y = df_train['target'].values
print(f"Number of Features: {len(feature_names)}")

Number of Features: 138


In [8]:
X[np.isnan(X)]=0.0
X[np.isinf(X)]=1e6

In [9]:
from sklearn.preprocessing import StandardScaler

def train_model(X, y, n_splits=5, tr_folds=[4]):
    models = []
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # time series split to find the best hyper-parameters
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        if fold not in tr_folds:
            continue
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # CatBoost model
        print(f"Training with CatBoost: Fold {fold}")
        model_cbt = cbt.CatBoostRegressor(loss_function='MAE',
                                          eval_metric = 'MAE',
                                          iterations=1000, 
                                          learning_rate=5e-3, 
                                          early_stopping_rounds=50,
                                          task_type="GPU",
                                          devices='0',
                                          boosting_type='Plain',
                                          l2_leaf_reg=10,
                                          thread_count=-1,
                                          verbose=True)
                        
        model_cbt.fit(X_train, y_train, eval_set=[(X_test, y_test)],  verbose=50)
        models.append(model_cbt)
        
        yval_cat_pred = model_cbt.predict(X_test)
        
        # XGBoost model
        print(f"Training with XGBoost: Fold {fold}")
        model_xgb = xgb.XGBRegressor(objective='reg:absoluteerror',
                                     eval_metric='mae',
                                     device='cuda',
                                     n_estimators=500, 
                                     learning_rate=0.05, 
                                     early_stopping_rounds=50,
                                     missing=0)
        scaler = StandardScaler()
        X_tr_scaled = scaler.fit_transform(X_train)
        model_xgb.fit(X_tr_scaled, y_train, eval_set=[(X_test, y_test)] , verbose=50)
        models.append(model_xgb)
        
        yval_xgb_pred = model_xgb.predict(X_test)
        best_loss, best_ws = 10, None
        
        # One can of course easily solve the best weights for the two models based on the validation set.
        # However, we manually choose these candidate weights to make the ensemble robust.
        for ws in [(.5,.5), (.6,.4), (.4,.6), (.7, .3), (.3, .7)]:
            yval_pred = ws[0]*yval_cat_pred + ws[1]*yval_xgb_pred
            loss_val = np.mean(np.abs(yval_pred - y_test))
            print(f"Using weight {ws} for Cat and XGB, val loss is {loss_val:.3f}")
            if loss_val < best_loss:
                best_loss = loss_val
                best_ws = ws

        

    return models, best_ws

In [10]:
# Train the models
models, ws = train_model(X, y)

Training with CatBoost: Fold 4


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 6.4907629	test: 5.9865484	best: 5.9865484 (0)	total: 22.5s	remaining: 6h 15m 21s
50:	learn: 6.4560204	test: 5.9593737	best: 5.9593737 (50)	total: 25.1s	remaining: 7m 47s
100:	learn: 6.4329070	test: 5.9418155	best: 5.9418155 (100)	total: 27.7s	remaining: 4m 6s
150:	learn: 6.4174313	test: 5.9302809	best: 5.9302809 (150)	total: 30.3s	remaining: 2m 50s
200:	learn: 6.4068221	test: 5.9225099	best: 5.9225099 (200)	total: 32.9s	remaining: 2m 10s
250:	learn: 6.3990882	test: 5.9168156	best: 5.9168156 (250)	total: 35.5s	remaining: 1m 45s
300:	learn: 6.3932988	test: 5.9130223	best: 5.9130223 (300)	total: 38.1s	remaining: 1m 28s
350:	learn: 6.3889120	test: 5.9102347	best: 5.9102347 (350)	total: 40.8s	remaining: 1m 15s
400:	learn: 6.3854013	test: 5.9080989	best: 5.9080989 (400)	total: 43.4s	remaining: 1m 4s
450:	learn: 6.3823250	test: 5.9061687	best: 5.9061687 (450)	total: 46.1s	remaining: 56.1s
500:	learn: 6.3794924	test: 5.9042105	best: 5.9042105 (500)	total: 48.7s	remaining: 48.6s
550:	

In [11]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [12]:
def inf(X):
    ypred = np.zeros(X.shape[0])
    X[np.isnan(X)]=0.0
    X[np.isinf(X)]=1e6
    for m, w in zip(models, ws):
        ypred += m.predict(X) * w
    return ypred
        

In [13]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))

    df_test, feature_names = generate_features(test)
    X_test = df_test[feature_names].values
    y_pred = inf(X_test)
    sample_prediction['target'] = y_pred
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
   stock_id  date_id  seconds_in_bucket  imbalance_size  \
0         0      478                  0      3753451.43   
1         1      478                  0       985977.11   
2         2      478                  0       599128.74   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                       -1         0.999875   11548975.43        NaN   
1                       -1         1.000245    3850033.97        NaN   
2                        1         1.000584    4359198.25        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap   row_id  \
0         NaN   0.999875  22940.00   1.000050   9177.60  1.0  478_0_0   
1         NaN   0.999940   1967.90   1.000601  19692.00  1.0  478_0_1   
2         NaN   0.999918   4488.22   1.000636  34955.12  1.0  478_0_2   

   currently_scored  
0             False  
1           